# Resumen de curso de ChatGPT Prompt Eng
Esto es un resumen de los ejemplos expuestos en el curso de ChatGPT Prompt Eng.
## Índice
1. [Principios de prompting y buenas prácticas](#principios-de-prompting)
2. [Ejemplos: Uso iterativo, resumen, inferencia, tranformación, expansión](#Examples)
3. [ChatBot](#ChatBot)

In [4]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

#### Función get_completion
Usaremos el `gpt-3.5-turbo` model y el [chat completions endpoint](https://platform.openai.com/docs/guides/chat). Una api que nos permite hacer preguntas y obtener respuestas.

Esta función va a ayudar a hacer prompts y mirar las salidas

In [5]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Principios de prompting
- **Principio 1: Instrucciones claras y específicas**
- **Principle 2: Hay que darle al modelo tiempo para pensar**

### Tácticas

#### Táctica 1: Utilizar delimitadores para indicar claramente partes distintas de la entrada
- Los delimitadores pueden ser cualquier cosa como:  ```, """, < >, `<tag> </tag>`, `:`

In [6]:
texto = f"""
Debes expresar lo que deseas que un modelo haga \ 
proporcionando instrucciones que sean lo más claras y \ 
específicas posible. \ 
Esto guiará al modelo hacia la salida deseada, \ 
y reducirá las posibilidades de recibir respuestas irrelevantes \ 
o incorrectas. No confundas escribir una \ 
indicación clara con escribir una indicación corta. \ 
En muchos casos, las indicaciones más largas proporcionan más claridad \ 
y contexto al modelo, lo que puede llevar a \ 
salidas más detalladas y relevantes.
"""

prompt = f"""
Resume el texto delimitado por triples comillas \ 
en una sola oración.
```{texto}```
"""
respuesta = get_completion(prompt)
print(respuesta)


Debes proporcionar instrucciones claras y específicas para guiar al modelo hacia la salida deseada y evitar respuestas irrelevantes o incorrectas, aunque las indicaciones más largas pueden proporcionar más claridad y contexto para obtener salidas más detalladas y relevantes.


#### Táctica 2: Solicitar una respuesta estructurada
- JSON, HTML

In [7]:
prompt = f"""
Genera una lista de tres títulos ficticios de libros, \ 
junto con sus autores y géneros. 
Proporciónalos en formato JSON con las siguientes claves: 
book_id, título, autor, género.
"""
respuesta = get_completion(prompt)
print(respuesta)

{
  "book_id": 1,
  "título": "El misterio del faro",
  "autor": "Laura García",
  "género": "Misterio"
},
{
  "book_id": 2,
  "título": "El secreto de la montaña",
  "autor": "Carlos Martínez",
  "género": "Aventura"
},
{
  "book_id": 3,
  "título": "La magia de los sueños",
  "autor": "María López",
  "género": "Fantasía"
}


#### Táctica 3: Solicitarle al modelo que verifique que se satisfagan condiciones

In [10]:
texto_1 = f"""
¡Preparar una taza de té es fácil! En primer lugar, necesitas \ 
poner a hervir un poco de agua. Mientras eso sucede, \ 
toma una taza y coloca una bolsita de té en ella. Una vez que el agua esté \ 
suficientemente caliente, simplemente viértela sobre la bolsita de té. \ 
Deja que repose un poco para que el té pueda infusionarse. Después de unos \ 
minutos, saca la bolsita de té. Si lo deseas, puedes añadir azúcar o leche al gusto. \ 
¡Y eso es todo! Tienes una deliciosa \ 
taza de té para disfrutar.
"""

texto_2 = f"""
El sol brilla intensamente hoy, y los pájaros están \
cantando. Es un hermoso día para dar un \ 
paseo por el parque. Las flores están floreciendo, y los \ 
árboles se mecen suavemente con la brisa. La gente \ 
está afuera, disfrutando del hermoso clima. \ 
Algunos están teniendo picnics, mientras que otros están jugando \ 
juegos o simplemente relajándose en el césped. Es un \ 
día perfecto para pasar tiempo al aire libre y apreciar la \ 
belleza de la naturaleza.
"""

prompt = f"""
Se te proporcionará un texto delimitado por triples comillas. 
Si contiene una secuencia de instrucciones, \ 
reescríbelas en el siguiente formato:

Paso 1 - ...
Paso 2 - …
…
Paso N - …

Si el texto no contiene una secuencia de instrucciones, \ 
simplemente escribe "No se proporcionaron pasos".

\"\"\"{texto_1}\"\"\"
"""

print("Completado para Texto 1:")
print(get_completion(prompt))

print("\nCompletado para Texto 2:")
print(get_completion(prompt.replace(texto_1, texto_2)))


Completado para Texto 1:
Paso 1 - Poner a hervir un poco de agua.
Paso 2 - Tomar una taza y colocar una bolsita de té en ella.
Paso 3 - Verter el agua caliente sobre la bolsita de té.
Paso 4 - Dejar que el té repose para que se infusione.
Paso 5 - Sacar la bolsita de té.
Paso 6 - Opcionalmente, añadir azúcar o leche al gusto.
Paso 7 - Disfrutar de una deliciosa taza de té.

 Completado para Texto 2:
No se proporcionaron pasos.


#### Táctica 4: "Few-shot" prompting (pocas muestras)

In [11]:
prompt = f"""
Tu tarea es responder de manera coherente.

<niño>: Enséñame sobre la paciencia.

<abuelo>: El río que talla el valle más profundo fluye desde un manantial modesto; \ 
la sinfonía más grandiosa tiene su origen en una sola nota; \ 
el tapiz más intrincado comienza con un solo hilo.

<niño>: Enséñame sobre la resiliencia.
"""
print(get_completion(prompt))


<abuelo>: La resiliencia es como un árbol que se dobla con el viento, pero nunca se quiebra. Es la capacidad de enfrentar los desafíos y superar las adversidades. Al igual que el árbol, podemos aprender a adaptarnos y encontrar fuerza en medio de las dificultades. La resiliencia nos enseña a levantarnos una y otra vez, incluso cuando pareciera que todo está en contra nuestra. Es una cualidad valiosa que nos permite crecer y transformarnos en personas más fuertes y sabias.
